# TP 4248 2021-2022 
## les membres du groupe
- fezeu Ghomsi Eugene Clotaire 17Q2864 
- Nono Mabou Wilfried Brondon 21S2817 brondonnono3@gmail.com
- NGBAYAFOU NGOUH Chéripha Cheriphalynette@gmail.com
- Junior Libii junior.libii@facsciences-uy1.cm

## binary naive bayes classification on sms dataset


In [41]:

import pandas as pd
import re
import numpy as np

dataset = pd.read_csv("./datas/spam.csv", encoding="ISO-8859-1", usecols=['v1', 'v2'], dtype={'v1':str,'v2':str})
dataset.head()
dataset = dataset.dropna(how="any")




In [35]:
!pip install nltk
import nltk
nltk.download('stopwords')
# ensemble de mots sans sens pas utilise pour la pertinence
# cette suppression offre un gain temps 
stopwords = nltk.corpus.stopwords.words('english')

#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()





[nltk_data] Downloading package stopwords to /home/eugene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
def tonkenizer(text):
    return re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()).split()
def remove_stopwords(text):
    return[i for i in text if i not in stopwords]
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

def cleantext (text):
    return stemming(remove_stopwords(tonkenizer(text)))
def clean_and_binarise_text(row):
    return   [(row[0] == 'ham' and 1) or 0 , cleantext(row[1])]    

dataset_r = dataset.apply(clean_and_binarise_text, axis=1, result_type='broadcast')
dataset_r

/home/eugene/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,v1,v2
0,1,"[go, jurong, point, crazi, avail, bugi, n, gre..."
1,1,"[ok, lar, joke, wif, u, oni]"
2,0,"[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,1,"[u, dun, say, earli, hor, u, c, alreadi, say]"
4,1,"[nah, think, goe, usf, live, around, though]"
...,...,...
5567,0,"[2nd, time, tri, 2, contact, u, u, 750, pound,..."
5568,1,"[b, go, esplanad, fr, home]"
5569,1,"[piti, mood, suggest]"
5570,1,"[guy, bitch, act, like, interest, buy, someth,..."
